In [ ]:
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt
from torchvision import transforms
import os

In [ ]:
path1 = "dog.jpg"
path2 = "dog2.jpg"

In [ ]:
image = Image.open("dog.jpg")

In [ ]:
print(image.height)
print(image.width)
print(image.size)
plt.imshow(image)

In [ ]:
#리사이즈

transform = transforms.Compose(
    [
        transforms.Resize(size=(32,32)),
        transforms.ToTensor()
    ]
)

transformed_image = transform(image)
plt.imshow(transformed_image.permute(1, 2, 0))

In [ ]:
#회전, 뒤집기

transform = transforms.Compose(
    [
        transforms.RandomRotation(degrees=30, expand=False, center=None),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5)
    ]
)

transformed_image = transform(image)
plt.imshow(transformed_image)

In [ ]:
# 원본 이미지 크기 얻기
width, height = image.size

# 절반 크기로 자르기
crop_size = (height // 2, width // 2)

transform = transforms.Compose(
    [
        transforms.CenterCrop(size=crop_size),
        transforms.Pad(padding=50, fill=(127, 127, 255), padding_mode="constant")
    ]
)
transformed_image = transform(image)
plt.imshow(transformed_image)

In [ ]:
transform = transforms.Compose(
    [
        # 각도(degrees), 
        # 이동(translate)
        # 척도(scale)
        # 전단(shear)
        transforms.RandomAffine(
            degrees=30, translate=(0.2, 0.2),
            scale=(0.8, 1.2), shear=15
        )
    ]
)
transformed_image = transform(image)
plt.imshow(transformed_image)

In [ ]:
transform = transforms.Compose(
    [
        transforms.ColorJitter(
            brightness=0.3, contrast=0.3,
            saturation=0.3, hue=0.3
        ),
        transforms.ToTensor(),
        transforms.Normalize(
            mean = [0.485, 0.456, 0.406],
            std = [0.229, 0.224, 0.225]
        ),
        transforms.ToPILImage()
    ]
)
transformed_image = transform(image)
plt.imshow(transformed_image)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomErasing(p=1.0, value=0),
    transforms.RandomErasing(p=1.0, value='random'),
    transforms.ToPILImage()
])

transformed_image = transform(image)
plt.imshow(transformed_image)

In [ ]:
import numpy as np
path = "dog2.jpg"
class Mixup:
    def __init__(self, target, scale, alpha=0.5, beta=0.5):
        self.target = target
        self.scale = scale
        self.alpha = alpha
        self.beta = beta

    def __call__(self, image):
        image = np.array(image)
        target = self.target.resize(self.scale)
        target = np.array(target)
        mix_image = image * self.alpha + target * self.beta
        return Image.fromarray(mix_image.astype(np.uint8))


transform = transforms.Compose(
    [
        transforms.Resize((512, 512)),
        Mixup(
            target=Image.open(path),
            scale=(512, 512),
            alpha=0.5,
            beta=0.5
        )
    ]
)
transformed_image = transform(image)
plt.imshow(transformed_image)

In [ ]:
import torch

def mixup(img1, img2, alpha=0.5):
    """
    두 이미지를 alpha, 1-alpha 비율로 Mixup하는 함수다.
    img1, img2는 torch.Tensor 형태(C, H, W)여야 한다.
    alpha는 0~1 사이의 float 값이다.
    """
    return img1 * alpha + img2 * (1 - alpha)


In [ ]:
import torch
from torchvision import transforms
from torchvision.transforms.v2 import MixUp
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

image1 = Image.open(path1)
image2 = Image.open(path2)

# 리사이즈 및 텐서 변환 transform 정의
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# 이미지 변환
img1 = transform(image1)
img2 = transform(image2)

# 배치 형태로 묶기 (B, C, H, W)
images = torch.stack([img1, img2], dim=0)

# 더미 레이블 생성 (MixUp은 레이블도 필요)
labels = torch.tensor([0, 1])

# MixUp transform 정의 (alpha=0.5 예시, num_classes=2)
mixup = MixUp(alpha=0.5, num_classes=2)

# MixUp 적용
mixed_images, mixed_labels = mixup(images, labels)

# 첫 번째 MixUp 결과 이미지만 시각화 (배치에서 하나 선택)
mixup_img_np = mixed_images[0].permute(1, 2, 0).numpy()
mixup_img_np = np.clip(mixup_img_np, 0, 1)

plt.imshow(mixup_img_np)
plt.axis('off')
plt.show()